In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "cluster-udacity"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster in the provided workspace, so we use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=3)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler ### YOUR CODE HERE ###
ps = RandomParameterSampling( {
    "--C" : uniform(0.0,1.0),
    "--max_iter" : choice(10,20,30,40,50,60,70,80,90,100)
    }
)

# Specify a Policy ### YOUR CODE HERE ###
policy = BanditPolicy(slack_factor = 0.1, 
                      evaluation_interval = 2,
                      delay_evaluation = 10)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
#src = ### YOUR CODE HERE ###
src = ScriptRunConfig(source_directory='.',
                            command=['python', 'train.py'],
                            compute_target=cluster_name,
                            environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy. ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(run_config=src, 
                             hyperparameter_sampling=ps,
                             policy=policy,
                             primary_metric_name='Accuracy',
                             primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                             max_total_runs = 10,
                             max_concurrent_runs = 2)



In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config=hyperdrive_config) 

#show run details with the widget.
RunDetails(hyperdrive_run).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = hyperdrive_run.get_best_run_by_primary_metric()
if best_run is None:
    raise Exception("No best run was found")
best_run

model_name = best_run.get_file_names()
file_name = "outputs/" + model_name

parameter_values = best_run.get_details()['runDefinition']['arguments']
best_parameters = dict(zip(parameter_values[::2], parameter_values[1::2]))

print(best_parameters)
    
if not os.path.exists('outputs/')):
    os.makedirs('outputs/')
joblib.dump(value=best_run, filename=file_name)

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
initial_data = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(initial_data)### YOUR DATA OBJECT HERE ###)

#Split data into train and test sets.
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    task='classification',
    primary_metric = "accuracy",
    training_data= [x_train,y_train],
    label_column_name=y,
    n_cross_validations=4)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###
run = exp.submit(automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run = run.get_best_child()

model_name = best_run.properties['model_name']
description = 'AutoML forecast example'
tags = None

model = run.register_model(model_name = model_name, 
                                  description = description, 
                                  tags = tags)